In [31]:
import os 
import pandas as pd
from tqdm import tqdm

from src.data_types import TOB, Order
from src.exchange import TOB_Exchange

%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# define folder path
folder_path = 'C:/Users/gobel/code/research_env/keyrock/bi-repo/data/tardis/datasets/'
# folder_path = '/Users/nick-kokkaliaris/Desktop/Keyrock/Keyrock/Tardis Playground/datasets/'
files_list = [
    f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
    and f.split('_')[0] == 'binance' 
    and f.split('_')[-1] == 'COMPBTC.csv.gz'
    #and f.split('_')[-2] == 'OG'
    and f.split('_')[1] == 'quotes']
files_list

['binance_quotes_2023-07-01_COMPBTC.csv.gz',
 'binance_quotes_2023-07-02_COMPBTC.csv.gz',
 'binance_quotes_2023-07-03_COMPBTC.csv.gz',
 'binance_quotes_2023-07-04_COMPBTC.csv.gz']

In [3]:
bin_trades = [
    f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
    and f.split('_')[0] == 'binance' 
    and f.split('_')[-1] == 'COMPBTC.csv.gz'
    #and f.split('_')[-2] == 'OG'
    and f.split('_')[1] == 'trades']
u_trades = [
    f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
    and f.split('_')[0] == 'upbit' 
    and f.split('_')[-1] == 'BTC-COMP.csv.gz'
    #and f.split('_')[-2] == 'OG'
    and f.split('_')[1] == 'trades']
ufiles_list = [
    f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) 
    and f.split('_')[0] == 'upbit' 
    and f.split('_')[-1] == 'BTC-COMP.csv.gz'
    #and f.split('_')[-2] == 'OG'
    and f.split('_')[1] == 'quotes']

In [4]:
def get_data(files_list):
    df = pd.DataFrame()
    # Use a loop to get all the csv.gz files from the path location in a single Dataframe
    for csv_file in tqdm(files_list):
        file_name = (folder_path + csv_file)
        if file_name[-2:] == "gz":
            df1 = pd.read_csv(file_name, compression='gzip')
            # Concatenate all the dataframes into a single dataframe
            df = pd.concat([df, df1], ignore_index=True)
    return df

In [5]:
upbit = get_data(ufiles_list)
binance = get_data(files_list)

upbit_t = get_data(u_trades)
binance_t = get_data(bin_trades)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 13.58it/s]


In [6]:
binance['time'] = pd.to_datetime(binance.timestamp, unit='us')
upbit['time'] = pd.to_datetime(upbit.timestamp, unit='us')
binance_t['time'] = pd.to_datetime(binance_t.timestamp, unit='us')
upbit_t['time'] = pd.to_datetime(upbit_t.timestamp, unit='us')

binance.set_index('time', inplace=True)
upbit.set_index('time', inplace=True)
binance_t.set_index('time', inplace=True)
upbit_t.set_index('time', inplace=True)

In [7]:
binance_t = binance_t[['timestamp', 'side', 'price', 'amount']].groupby(['timestamp', 'side', 'price']).sum().reset_index()

In [8]:
binance_t

,timestamp,side,price,amount
0,1688169650044000,buy,0.001837,5.242
1,1688169650454000,sell,0.001836,8.937
2,1688169650454000,sell,0.001837,17.866
3,1688169662969000,buy,0.001838,21.387
4,1688169691584000,sell,0.001835,2.262
...,...,...,...,...
46438,1688515122993000,buy,0.002031,1.981
46439,1688515170284000,sell,0.002030,24.128
46440,1688515176781000,buy,0.002032,2.135
46441,1688515178034000,buy,0.002033,11.198


Updates: 
- 1: bid quantity update 
- 2: new top bid 
- 3: new top ask 
- 4: ask quantity update 
- 5: complete update

In [9]:
lst = binance[['timestamp', 'bid_amount', 'bid_price', 'ask_price', 'ask_amount']].sort_values('timestamp').to_numpy()

In [10]:
exchg = TOB_Exchange()

In [19]:
exchg.load_tob(lst, 'COMPBTC')

In [20]:
t = exchg.events.peekitem(0)[1].popleft()

In [21]:
exchg.events.peekitem(0)[1]

deque([])

In [15]:
w = exchg.top_of_book('COMPBTC')

Bid: 0.001833 | 0.001834 :Ask


In [42]:
current = lst[0]
current = TOB(timestamp=current[0], bq=current[1], bp=current[2], ap=current[3], aq=current[4])

In [43]:
n = lst[1]
n = TOB(timestamp=n[0], bq=n[1], bp=n[2], ap=n[3], aq=n[4])

In [49]:
c = 0

if current.bq != n.bq: 
    c += 1
    print('change in bid quantity')
    if current.bp != n.bp:
        c += 1
        print('also price change')
elif current.aq != n.aq:
    print('change in ask quantity')

    

change in bid quantity


In [48]:
n

TOB(timestamp=1688169604648000.0, bq=11.185, bp=0.001833, ap=0.001834, aq=2.264)

In [45]:
current

TOB(timestamp=1688169604555574.0, bq=21.861, bp=0.001833, ap=0.001834, aq=2.264)

In [ ]:
def strategy(top_bid, top_ask):
    
    if open_orders:

In [17]:
upbit_t

,exchange,symbol,timestamp,local_timestamp,id,side,price,amount
time,,,,,,,,
2023-07-01 00:00:04.991,upbit,BTC-COMP,1688169604991000,1688169605052807,16881696049910000,buy,0.001838,0.435206
2023-07-01 00:00:33.090,upbit,BTC-COMP,1688169633090000,1688169633143121,16881696330900000,buy,0.001838,11.569343
2023-07-01 00:00:51.615,upbit,BTC-COMP,1688169651615000,1688169651683521,16881696516150000,buy,0.001838,0.272007
2023-07-01 00:01:38.951,upbit,BTC-COMP,1688169698951000,1688169699023291,16881696989510000,sell,0.001816,3.145549
2023-07-01 00:01:50.078,upbit,BTC-COMP,1688169710078000,1688169710144191,16881697100780000,buy,0.001838,0.275292
...,...,...,...,...,...,...,...,...
2023-07-04 23:55:51.298,upbit,BTC-COMP,1688514951298000,1688514951363019,16885149512980002,sell,0.002023,4.327845
2023-07-04 23:55:51.298,upbit,BTC-COMP,1688514951298000,1688514951363020,16885149512980004,sell,0.002011,0.452033
2023-07-04 23:58:19.507,upbit,BTC-COMP,1688515099507000,1688515099572823,16885150995070000,sell,0.002012,3.253482
